In [4]:
from itertools import islice

import matplotlib.dates as mdates
import pandas as pd
import torch
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.common import ListDataset
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation import Evaluator, make_evaluation_predictions
from lag_llama.gluon.estimator import LagLlamaEstimator
from matplotlib import pyplot as plt

from tqdm.autonotebook import tqdm

import yfinance as yf

In [6]:
data = yf.download("AAPL", start="2010-01-01", end="2023-12-31")

# Preproses data (contoh sederhana)
train_data = data.iloc[:int(len(data)*0.7)]
val_data = data.iloc[int(len(data)*0.7):int(len(data)*0.9)]
test_data = data.iloc[int(len(data)*0.9):]


# Buat dataset Gluonts
train_ds = ListDataset([{"start": train_data.index[0], "target": train_data["Close"]}], freq="D")
val_ds = ListDataset([{"start": val_data.index[0], "target": val_data["Close"]}], freq="D")
test_ds = ListDataset([{"start": test_data.index[0], "target": test_data["Close"]}], freq="D")


[*********************100%***********************]  1 of 1 completed


In [8]:
prediction_length = 60
num_samples = 1060
device = torch.device("cuda:0")

ckpt = torch.load("./lag-llama-model/lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

rope_scaling_arguments = {
    "type": "linear",
    "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
}
estimator = LagLlamaEstimator(
    ckpt_path="./lag-llama-model/lag-llama.ckpt",
    prediction_length=prediction_length,
    context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length
    # estimator args
    input_size=estimator_args["input_size"],
    n_layer=estimator_args["n_layer"],
    n_embd_per_head=estimator_args["n_embd_per_head"],
    n_head=estimator_args["n_head"],
    scaling=estimator_args["scaling"],
    time_feat=estimator_args["time_feat"],
    rope_scaling=rope_scaling_arguments if use_rope_scaling else None,
    batch_size=1,
    num_parallel_samples=100,
    device=device,
)
lightning_module = estimator.create_lightning_module()
transformation = estimator.create_transformation()
predictor = estimator.create_predictor(transformation, lightning_module)
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset,
    predictor=predictor,
    num_samples=num_samples
)
forecasts = list(forecast_it)
tss = list(ts_it)

[{'start': Period('2010-01-04', 'D'),
  'target': array([ 7.643214,  7.656429,  7.534643, ..., 58.83    , 58.5925  ,
         58.82    ], dtype=float32)}]